## NYCbuswatcher
#### Step 1. Convert JSON shipments --> parquets using Pyarrow

In [1]:
import os, json
import pandas as pd
import numpy as np

In [2]:
in_path="/Volumes/nice/bigdata/bus_depot/bus-data-migrations/nyc/data/in/nyc_shipments/2022/2/2"

shipment_list = []
for root, _, filenames in os.walk(in_path):   
    for filename in filenames:
        shipment_list.append(os.path.join(root, filename))
len(shipment_list)

5832

In [3]:
shipment_list[0]

'/Volumes/nice/bigdata/bus_depot/bus-data-migrations/nyc/data/in/nyc_shipments/2022/2/2/0/BX16/shipment_2022-2-2-0-BX16.json'

In [4]:
out_path="/Volumes/nice/bigdata/bus_depot/bus-data-migrations/nyc/data/out/nyc_shipments_as_parquets/"
os.makedirs(out_path, exist_ok=False)

from pyspark.sql.types import IntegerType

for shipment in shipment_list:
    with open(shipment) as data_file:    
        data = json.load(data_file)
        try:
            df = pd.json_normalize(data, 'buses').convert_dtypes()
            # print(f"Success loading buses from {shipment}")
        except:
            # print(f"Error loading buses from {shipment}")
            pass

        # not sure why this is so hard, but need to ensure every file has the same schema
        
        df["passenger_count"] = pd.to_numeric(df["passenger_count"],errors='coerce')
        
        if df.dtypes['passenger_count'] != "float64":
            df['passenger_count'] = df['passenger_count'].astype(dtype = 'float64')
            
        if df.dtypes['bearing'] != "float64":
            df['bearing'] = df['bearing'].astype(dtype = 'float64')
            
        if df.dtypes['next_stop_d_along_route'] != "float64":
            df['next_stop_d_along_route'] = df['next_stop_d_along_route'].astype(dtype = 'float64')

        if df.dtypes['next_stop_d'] != "float64":
            df['next_stop_d'] = df['next_stop_d'].astype(dtype = 'float64')
            
        # write to disk
        out_file=out_path+shipment.split('/')[-1].split('.')[0]+".parquet"
        df.to_parquet(out_file, engine="fastparquet")


FileExistsError: [Errno 17] File exists: '/Volumes/nice/bigdata/bus_depot/bus-data-migrations/nyc/data/out/nyc_shipments_as_parquets/'